In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import PolynomialFeatures
from modelo_cuadratico import modelo_cuadratico

In [ ]:
run_test = False # Si está en True se va a hacer una prueba inicial con datos sintéticos
load_model = True # Si está en True no se va a entrenar y va a cargar los pesos load_weights_file
save_model = False # Si está en True se van a guardar los pesos luego de entrenar en save_weights_file
load_weights_file = "model.pt" # Archivo con los pesos guardados.
save_weights_file = "model.pt" # Archivo donde se guardan los pesos 

### Proto Recomendador
Esta notebook está basada en el paper A Recommendation Model Based on Deep Neural Network del autor LIBO ZHANG

#### Prueba con datos sintéticos

In [ ]:
if not run_test: print("run_test = False -> No se va a correr el test")

In [ ]:
if run_test:
    n_users = 50
    n_items = 100
    a = 10
    b = 20
    sparce_rate = .9 # Qué tan dispersa es la matriz de ratings de ejemplo

    # Inicializo la matriz de interacciones y pongo -1 en muchos lugares
    # El -1 representa las interacciones que no conocemos o no sucedieron. Ej: las peliculas que no vio el usuario.
    ratings = np.random.randint(0, 2, n_users*n_items).reshape(n_users,n_items)
    aux = np.random.rand(ratings.shape[0], ratings.shape[1])
    ratings[aux<sparce_rate] = -1
    ratings_df = pd.DataFrame(ratings)
    ratings_df.columns = list(ratings_df.columns + 1)
    ratings_df.index = list(ratings_df.index + 1)
    ratings = torch.from_numpy(ratings)
    
    model = modelo_cuadratico(a, b, n_users, n_items, ratings_df)

In [ ]:
%time if run_test: R = model.entrenar(ratings)

#### Prueba con datos de MovieLens

Uso el dataset de MovieLens 1M (https://grouplens.org/datasets/movielens/1m/).
Para correr esta notebook hay que bajarlo, descomprimirlo y apuntar los siguientes paths a los archivos correspondientes

In [ ]:
user_data = '../../MovieLens/data/users.dat'
movie_data = '../../MovieLens/data/movies.dat'
ratings_data = '../../MovieLens/data/ratings.dat'

In [ ]:
users = pd.read_csv(user_data, sep = '::', header = None, names = ['UserId', 'Gender', 'AgeGroup', 'Occupation', 'Zip'], encoding = 'latin-1')
movies = pd.read_csv(movie_data, sep = '::', header = None, names = ['MovieId', 'Title', 'Genre'], encoding = 'latin-1')
ratings_data = pd.read_csv(ratings_data, sep = '::', header = None, names = ['UserId', 'MovieId', 'Rating', 'Timestamp'], encoding = 'latin-1')

In [ ]:
ratings_data.head()

In [ ]:
ratings_data['Match'] = (ratings_data['Rating']>=4).astype(int) # Considero un match si el ratings es 4 o más (esto es bastante burdo pero es para probar)

In [ ]:
ratings_df = ratings_data.pivot(index='UserId',
                                 columns='MovieId',
                                 values='Match').fillna(-1)

In [ ]:
ratings_df.head()

In [ ]:
ratings_matrix = ratings_df.to_numpy()

In [ ]:
n_users = ratings_data.UserId.nunique()
n_items = ratings_data.MovieId.nunique()
a = 16
b = 20
l = a + b
ratings = ratings_matrix.copy()
ratings = torch.from_numpy(ratings)

In [ ]:
model = modelo_cuadratico(a, b, n_users, n_items, ratings_df)

### Entrenamiento Regresión Cuadrática

In [ ]:
if load_model: print("No se va a entrenar el modelo. Se va a cargar de {}".format(load_weights_file))

In [ ]:
if load_model:
    model.load_weights(load_weights_file)
else:
    %time R = model.entrenar(ratings, lr = 0.1)
    if save_weights:
        model.save_weights(save_weights_file)